In [3]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import copy
cfg = Config.fromfile('./configs/custom/final.py')
lr_c = ['step']
cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swinadamw20cas_aug2_trash'
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
#cfg.model.roi_head.bbox_head.num_classes = 10
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model,
                       train_cfg=cfg.get('train_cfg'),
                       test_cfg=cfg.get('test_cfg'))
model.init_weights()

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:915: FutureWarning: This class has been deprecated. Please use ImageCompression
  FutureWarning,
/opt/ml/detection/new_mmdetection/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-10-07 14:58:26,682 - mmdet - INFO - Use load_from_http loader
2021-10-07 14:58:26,845 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2021-10-07 14:58:26,870 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2021-10-07 14:58:26,879 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'lay

In [4]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2021-10-07 14:58:34,979 - mmdet - INFO - Start running, host: root@feced5044610, work_dir: /opt/ml/detection/new_mmdetection/work_dirs/swinadamw20cas_aug2_trash
2021-10-07 14:58:34,980 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook              

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:192: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')


wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/ml/detection/new_mmdetection/mmdet/datasets/custom.py", line 195, in __getitem__
    data = self.prepare_train_img(idx)
  File "/opt/ml/detection/new_mmdetection/mmdet/datasets/custom.py", line 218, in prepare_train_img
    return self.pipeline(results)
  File "/opt/ml/detection/new_mmdetection/mmdet/datasets/pipelines/compose.py", line 41, in __call__
    data 

/opt/conda/envs/detection/lib/python3.7/site-packages/albumentations/augmentations/functional.py:652: UserWarning: Image compression augmentation is most effective with uint8 inputs, float32 is used as input.
  UserWarning,


In [ ]:
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

In [ ]:
x = df['id']
y = df['category_id']
groups = df['image_id']
seed = 123456
k = 5

In [ ]:
# %%timeit
# 7.23 s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

labels_num = np.max(y) + 1  # label이 0~9로 구성이 되어 있어 최대 값에 1을 더해 label 개수를 파악합니다.
# 각 image 별로 label 개수를 확인하기 위해 지정합니다.
y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))  # dict shape: (n_image, n_label)
# 각 label 별 개수를 확인하기 위해 지정합니다.
y_distr = Counter()
# annotation 데이터를 이용해 순차적으로 데이터를 확인합니다.
for label, g in zip(y, groups):
    y_counts_per_group[g][label] += 1
    y_distr[label] += 1

y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))  # fold별 각 label 개수를 파악하기 위해 지정합니다.

# 어떤 image가 특정 fold에 속할 때 전체 fold의 std 값 확인하는 함수입니다.
def eval_y_counts_per_fold(y_counts, fold):
    y_counts_per_fold[fold] += y_counts  # image가 특정 fold에 속한 것을 반영합니다.
    std_per_label = []
    # label 별로 전체 fold에 대해 std 값을 확인합니다.
    for label in range(labels_num):
        # label 별로 개수가 다르기 때문에 scale을 맞추기 위해 각 label 별 총 개수로 나누어 줍니다.
        label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
        std_per_label.append(label_std)
    # 본 함수는 해당 image가 특정 fold에 속한 것을 가정하고 계산합니다. 해당 image가 특정 fold에 속하지 않을 수 있기 때문에 원상 복구 합니다.
    y_counts_per_fold[fold] -= y_counts
    # label별 std를 평균냅니다.
    return np.mea가(std_per_label)

# image당 각 label 개수에 대한 값들만 활용합니다.
groups_and_y_counts = list(y_counts_per_group.items())
# shuffle 후 sort 하는 것이 이상해 보이지만, sort 하였을 때 같은 std 값일 경우 순서가 달라지게 됩니다.
random.Random(seed).shuffle(groups_and_y_counts)
groups_per_fold = defaultdict(set)  # 각 fold별 index를 저장하기 위해 지정합니다.

# 큰 std를 가진 image부터 순차적으로 best fold를 찾습니다.
for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
    best_fold = None
    min_eval = None
    # image가 어떤 fold에 속할 때 가장 낮은 std를 가지는지 확인하는 과정입니다.
    for i in range(k):  # 각 fold에 대해 순차적으로 확인합니다.
        fold_eval = eval_y_counts_per_fold(y_counts, i)  # 전체 fold에 대해 std값 확인.
        # std 값이 가장 낮을 때가 best fold 입니다.
        if min_eval is None or fold_eval < min_eval:
            min_eval = fold_eval
            best_fold = i
    y_counts_per_fold[best_fold] += y_counts  # best fold를 확정하고 해당 image를 반영합니다.
    groups_per_fold[best_fold].add(g)  # best fold에 해당 이미지의 index를 추가합니다.

NameError: name 'y' is not defined

In [ ]:
# %%timeit
# 838 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

labels_num = y.max() + 1
# https://stackoverflow.com/a/39132900/14019325
# 기존 코드의 첫번째 loop와 동일합니다. 각 image 별 label 개수를 확인합니다.
y_counts_per_group = df.groupby(['image_id', 'category_id']).size().unstack(fill_value=0)
y_counts_per_fold = np.zeros((k, labels_num))

# scale을 미리 계산하여 연산을 줄입니다.
y_norm_counts_per_group = y_counts_per_group / y_counts_per_group.sum()
# suffle & sort
shuffled_and_sorted_index = y_norm_counts_per_group.sample(frac=1, random_state=seed).std(axis=1).sort_values(ascending=False).index
y_norm_counts_per_group = y_norm_counts_per_group.loc[shuffled_and_sorted_index]

groups_per_fold = defaultdict(set)

for g, y_counts in zip(y_norm_counts_per_group.index, y_norm_counts_per_group.values):
    best_fold = None
    min_eval = None
    for fold_i in range(k):
        # 기존 코드 eval_y_counts_per_fold 와 동일합니다.
        y_counts_per_fold[fold_i] += y_counts
        fold_eval = y_counts_per_fold.std(axis=0).mean()  # numpy를 활용하여 연산을 단순화 합니다.
        y_counts_per_fold[fold_i] -= y_counts
        if min_eval is None or fold_eval < min_eval:
            min_eval = fold_eval
            best_fold = fold_i
    y_counts_per_fold[best_fold] += y_counts
    groups_per_fold[best_fold].add(g)